In [1]:
import pandas as pd

# 1. File Reading: Read the file once and extract required sheets

xls = pd.ExcelFile("FantaSPL_V2.2.xlsx")
roles_df = xls.parse(sheet_name=3)
data_first_sheet = xls.parse(sheet_name=0)
punti_df = xls.parse(sheet_name="Punti")

# 2. Column Renaming and Data Transformation
# Provided dictionaries for translations
translation_dict = {
    "Difensore": "Defender",
    "Centrocampista": "Midfielder",
    "Attaccante": "Forward",
    "Portiere": "Goalkeeper"
}
column_mapping_first_sheet = {
    "Data": "Date",
    "Tipo Partita": "Match Type",
    "Bianchi": "Team A Goals",
    "Scuri": "Team B Goals",
    "Squadra vincente": "Winning Team",
    "Numero Goal": "Number of Goals",
    "Goal Check": "Goal Check",
    "Numero Goal Punti": "Goal Points",
    "Autogoal": "Own Goals",
    "Costo per Partita": "Cost per Match",
    "Effettivo": "Actual Cost"
}
translations = {
    "Data": "Date",
    "Player": "Player",
    "Ruolo": "Role",
    "Colore Squadra": "Team Color",
    "Vincitori": "Winners",
    "Vinta": "Won",
    "#Goal": "Goals",
    "Goal Subiti": "Goals Conceded",
    "Goal Fatti": "Goals Scored",
    "AutoGoal": "Own Goals",
    "SPL Bonus": "SPL Bonus",
    "MVP": "MVP",
    "Amici Portati": "Friends Brought",
    "Penalità": "Penalties",
    "Presenza": "Attendance",
    "Penalità.1": "Penalties.1",
    "Punteggio Difesa": "Defense Score",
    "Punteggio Centrocampisti": "Midfielders Score",
    "Squadra Vincente": "Winning Team",
    "#Goal.1": "Goals.1",
    "AutoGoal.1": "Own Goals.1",
    "SPL Bonus.1": "SPL Bonus.1",
    "MVP.1": "MVP.1",
    "Porta Amico": "Friend Referral",
    "TOTAL": "TOTAL"
}

# Merging the dictionaries for translations and column mapping
translations.update(translation_dict)
translations.update(column_mapping_first_sheet)

# Continue with the rest of the optimization
roles_df["Position"] = roles_df["Ruolo"].map(translations)
data_first_sheet.rename(columns=translations, inplace=True)
punti_df.rename(columns=translations, inplace=True)

# Transformations
user_input_table = data_first_sheet[["Date", "Match Type", "Team A Goals", "Team B Goals"]].copy()
user_input_table["Winning Team"] = user_input_table.apply(
    lambda row: "A" if row["Team A Goals"] > row["Team B Goals"] else ("B" if row["Team A Goals"] < row["Team B Goals"] else "X"), axis=1
)
user_input_table["Total Goals"] = user_input_table["Team A Goals"] + user_input_table["Team B Goals"]

# Further transformations for punti_df
user_input_df = punti_df[['Date', 'Player', 'Team Color', 'Goals', 'Own Goals', 'SPL Bonus', 'MVP', 'Friends Brought', 'Penalties']].copy()
user_input_df['Team Color'].replace({'Scuri': 'B', 'Bianchi': 'A'}, inplace=True)
replacement_dict = {'Sì': 1, 'No': 0}
for column in ['SPL Bonus', 'MVP', 'Penalties']:
    user_input_df[column].replace(replacement_dict, inplace=True)

# Efficient game_outcome function
def game_outcome(row):
    if row['Winning Team'] == 'X':
        return 'Draw'
    elif row['Team Color'] == row['Winning Team']:
        return 'Won'
    return 'Lost'

merged_table = user_input_df.merge(user_input_table, on='Date', how='inner')
merged_table['Game Outcome'] = merged_table.apply(game_outcome, axis=1)
merged_table['Goals Conceded'] = merged_table.apply(
    lambda row: row['Team B Goals'] if row['Team Color'] == 'A' else row['Team A Goals'],
    axis=1
)
merged_table['Goals Scored'] = merged_table.apply(
    lambda row: row['Team A Goals'] if row['Team Color'] == 'A' else row['Team B Goals'],
    axis=1
)
merged_table = merged_table.merge(roles_df[['Player', 'Position']], on='Player', how='left')
final_columns = ["Date", "Player", "Position", "Team Color", "Game Outcome", "Goals", "Own Goals", "Goals Conceded", "Goals Scored", "SPL Bonus", "MVP", "Friends Brought", "Penalties"]
merged_table = merged_table[final_columns]

def position_7(position_11):
    if position_11 == "Defender":
        return "Defensive"
    elif position_11 in ["Midfielder", "Forward"]:
        return "Offensive"
    elif position_11 == "Goalkeeper":
        return "Goalkeeper"

def position_5(position_11):
    if position_11 in ["Defender", "Midfielder", "Forward"]:
        return "Outfield"
    elif position_11 == "Goalkeeper":
        return "Goalkeeper"

roles_df['Position 7'] = roles_df['Position'].apply(position_7)
roles_df['Position 5'] = roles_df['Position'].apply(position_5)
merged_table = merged_table.merge(roles_df[['Player', 'Position 7', 'Position 5']], on='Player', how='left')

# Display the optimized merged table
merged_table.head(10)


,Date,Player,Position,Team Color,Game Outcome,Goals,Own Goals,Goals Conceded,Goals Scored,SPL Bonus,MVP,Friends Brought,Penalties,Position 7,Position 5
0,2023-01-12,Andrea Limonta,Defender,B,Lost,0,0,18,13,1,0,0,0,Defensive,Outfield
1,2023-01-12,Luca Stoppi,Defender,B,Lost,2,1,18,13,0,0,0,0,Defensive,Outfield
2,2023-01-12,Riccardo Ricci,Midfielder,B,Lost,0,0,18,13,0,0,0,0,Offensive,Outfield
3,2023-01-12,Marco Taglio,Midfielder,B,Lost,3,0,18,13,0,0,0,0,Offensive,Outfield
4,2023-01-12,Mazzu,Forward,B,Lost,7,0,18,13,0,0,0,0,Offensive,Outfield
5,2023-01-12,Rudy Vitiello,Midfielder,B,Lost,1,0,18,13,0,0,1,0,Offensive,Outfield
6,2023-01-12,Carlo Tempesta,Midfielder,B,Lost,0,0,18,13,0,0,0,1,Offensive,Outfield
7,2023-01-12,Fabrizio Limonta,Defender,A,Won,0,0,13,18,0,0,0,0,Defensive,Outfield
8,2023-01-12,Cameron McAinsh,Midfielder,A,Won,5,0,13,18,0,0,0,0,Offensive,Outfield
9,2023-01-12,Giovanni Aiello,Defender,A,Won,1,0,13,18,0,0,0,0,Defensive,Outfield
